#### Warm-up exercise to recall iterations

In [13]:
names = ["NGC 5128", "TXS 0506+056", "NGC 1068", "GB6 J1040+0617", "TXS 2226-184"]
distances = [3.7, 1.75e3, 14.4, 1.51e4, 107.1]  # Mpc
luminosities = [1e40, 3e46, 4.9e38, 6.2e45, 5.5e41] # erg/s

gal_cat = list(zip(names, distances, luminosities))

for name, dist, lum in gal_cat:
    print(f"{name:15s} D={dist:.2e} Mpc, L={lum:.2e} erg/s")

NGC 5128        D=3.70e+00 Mpc, L=1.00e+40 erg/s
TXS 0506+056    D=1.75e+03 Mpc, L=3.00e+46 erg/s
NGC 1068        D=1.44e+01 Mpc, L=4.90e+38 erg/s
GB6 J1040+0617  D=1.51e+04 Mpc, L=6.20e+45 erg/s
TXS 2226-184    D=1.07e+02 Mpc, L=5.50e+41 erg/s


# (Almost) all about functions
### 11/11/2022

In general, a function takes ***arguments*** as input, processes them, and ***returns*** a result as output 

In [37]:
from math import pi

# From here on, you're expected to document all your functions 
# with a docstring, like in the one below. You can read about 
# docstrings towards the end of this notebook.

def calc_flux(luminosity, distance_mpc):
    '''
    '''
    distance_cm = distance_mpc * 3e24
    flux = luminosity / (4 * pi * distance_cm ** 2)
    return flux

lum, dist = 4e45, 100

flux = calc_flux(lum, dist)

print(f"{flux:.2e} erg/s")

3.54e-09 erg/s


If you forget a `return` statement, your function will return `None`

In [17]:
def add_one(num):
    num += 1
    
print(add_one(10))

None


### Scope

In [18]:
count = 0

def bad_function():
    count += 1

# stepup() # Calling this function will return an error

Variables defined inside the namespace of a function are *local* to that function.

Variables created outside a function are called *global* variables

In [19]:
def add_one(count):
    count += 1 # count is a local variable because it's an argument
    return count


count = 1      # global 
print(count)

add_one(count) # this is returning 2 and we're doing nothing with the result
print(count)   # global variable is unaffected

count = add_one(count) # only now are we updating the variable count 
print(count)

1
1
2


Best practice: call differently your arguments, local variables and global variables

In [20]:
# Exercise
def add_one(n):
    res = n + 1  # Both n and res are local variables
    return res

count = 1      # Calling our global and local variables differently
               # avoids confusion
print(count)
count = add_one(count) # this returns 2 and we're replacing our global variable with it
print(count)   # global variable is changed


1
2


A function can return *any* type, including lists, dictionaries, booleans, or even functions

In [21]:
def is_detectable(flux):
    return flux > 1e-11

print(is_detectable(1e-12))


False


In [22]:
# Exercise:
def is_detectable(luminosity, distance):
    flux = calc_flux(luminosity, distance)
    return flux > 1e-11

for name,dis,lum in gal_cat:
    if is_detectable(lum, dis):
        print(f"{name:15s} is detectable")
    else:
        print(f"{name:15s} is not detectable")

NGC 5128        is not detectable
TXS 0506+056    is detectable
NGC 1068        is not detectable
GB6 J1040+0617  is not detectable
TXS 2226-184    is not detectable


A function termintes the first time that `return` is called - beware of pitfalls!

In [23]:
def find_first_detectable(catalog):
    for name, dis, lum in catalog:
        if is_detectable(lum, dis):
            return name

firstname = find_first_detectable(gal_cat)
print(f"First resolved galaxy: {firstname}")  # I can do this
# print(f"First resolved galaxy: {firstname:s}") # But I wouldn't be able to do this

First resolved galaxy: TXS 0506+056


The problem with the above function is that if there are no elements that satisfy our requirement, the `return` statement will never be called and the function will return a `NoneType`. Let's fix that:

In [24]:
# Exercise: fix the above function

def find_first_detectable(catalog):
    firstname = "None!"
    for name, dis, lum in catalog:
        if is_detectable(lum, dis):
            firstname = name
    return firstname

firstname = find_first_detectable(gal_cat)
print(f"First resolved galaxy: {firstname:s}") # Now I know that a string will always be returned

First resolved galaxy: TXS 0506+056


Python functions are extremely flexible and can even return multiple variables of different types

In [25]:
def assess_flux(luminosity, distance):
    flux = calc_flux(luminosity, distance)
    isdetect = is_detectable(luminosity, distance)
    return flux, isdetect

results  = assess_flux(1e45, 100)
print(results)

if results[0]:
    print("A flux of {results[1]:.2e} erg/cm2/s is detectable!\n")

# A better syntax is to "unpack" the result into different variables:

flx, isdet = assess_flux(1e45, 100)

print(flx, isdet)

if isdet:
    print("A flux of {flx:.2e} erg/cm2/s is detectable!\n")

(8.841941282883073e-10, True)
A flux of {results[1]:.2e} erg/cm2/s is detectable!

8.841941282883073e-10 True
A flux of {flx:.2e} erg/cm2/s is detectable!



### Keyword arguments

We can give some arguments default values. These are called *keyword* arguments, while the ones that are not defaulted are called *positional* arguments

In [26]:
# Exercise:
def is_detectable(luminosity, distance, threshold=1e-11): # luminosity and distance are positional: 
                                                          # they must always be passed then calling 
                                                          # the function. threshold is keyword, and will
                                                          # be defaulted to 1e-11 if I don't pass it
                                                          # to the function
                                                        
    flux = calc_flux(luminosity, distance)
    return flux > threshold

is_detectable(1e45,100) # I don't give any value of threshold,
                        # so Python assumes the default value 
                        # I defined in the function (in this case 1e-11) 
        
is_detectable(1e45,100, 1e-12) # Now Python takes  the default value 
                        # I defined in the function 
        

True

These defaulted parameters must come ***after*** all the undefined arguments

In [27]:
# Trying to define a function like this will throw an error:

# def is_detectable(luminosity, threshold=1e-11, distance):
#     flux = calc_flux(luminosity, distance)
#     return flux > threshold

When you add a parameter to a function, always remember to update all the functions that depend on it! 

In [28]:
# Exercise: 
def find_first_detectable(catalog, threshold=1e-11):
    firstname = "None!"
    for name, dis, lum in catalog:
        if is_detectable(lum, dis, threshold): # I pass on the threshold
                                               # parameter to all functions
                                               # that depend on it
            firstname = name
    return firstname

## Recursion

Functions can not only depend on other functions, but also on themselves. In curs

In [32]:
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)
    
for i in range(12):
    print(fibonacci(i))

0
1
1
2
3
5
8
13
21
34
55
89


In [30]:
# Exercise
def factorial(n):
    if n <= 1:
        return 1
    else:
        return n * factorial(n-1)

for i in range(10):
    print(i, factorial(i))

0 1
1 1
2 2
3 6
4 24
5 120
6 720
7 5040
8 40320
9 362880


### Defining functions in one line with `lambda`

In [35]:
# I can also write the above function as

fibonacci = lambda n: n * factorial(n-1) if n > 1 else 1

print(fibonacci(4.), fibonacci(10))

# lambda functions can also take several arguments
# (but should be used only for simple tasks)

hypothenuse = lambda x,y: (x ** 2 + y ** 2) ** 0.5

sa, sb = 3, 4
sc = hypothenuse(sa,sb)
print(f"A={sa}, B={sb} -> C = {sc}")

24.0 3628800
A=3, B=4 -> C = 5.0


### Applying functions to sequences with `map()`  

In [12]:
xvals = list(range(11))

f1 = lambda x: x ** 3
yvals = list(map(f1, xvals))

print (xvals)
print (yvals)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[0, 1, 8, 27, 64, 125, 216, 343, 512, 729, 1000]


In [39]:
# Exercise

calcflux = lambda row: calc_flux(row[2], row[1])
fluxes = list(map(calcflux, gal_cat))
print(fluxes)

[6.4586861087531586e-12, 8.661493501599748e-11, 2.089386202070171e-14, 2.404282090867728e-13, 4.2396633647669894e-13]


### Docstrings

Adding docstrings to your functions will help you become a better programmer and a better scientist!

Your future self and your collaborators will appreciate it, becuase your code will be clearer, which will help prevent errors in your results. (On top of this, your future employers will love seeing your neatly documented code on your GitHub repository) 

Even in the most obvious cases, your docstring should be at least one line: 

In [229]:
def add_one(n):
    """Calculate n+1 and return the result."""
    res = n + 1  
    return res

def check_script():
    """Check if the script is running."""

If the funtion does somthing more complex, you should write a more complere docstring, in this general form:

In [227]:
def my_function(par1, par2):
    """
    One-line description of the purpose of the function.

    If necessary, you can add here a second paragraph explaining in detail
    the rationale and usage of the function, including an example if 
    necessary. By using three quotation marks, every line in between is 
    interpreted as part of the same string. So use line breaks like this 
    to keep your lines short.
    
    Args:
        par1: a number
        par2: a second number
    
    Returns:
        The result of some operation on our input
    """
    res = some_operation(par1, par2)
    return res

Strings written in this fashion will become the docstring of the funcion, which will help you future self or your collaborators understand your code:

In [226]:
help(my_function)

Help on function my_function in module __main__:

my_function(par1, par2)
    One-line description of the purpose of the function.
    
    If necessary, you can add here a second paragraph explaining in detail
    the rationale and usage of the function, including an example if 
    necessary. By using three quotation marks, every line in between is 
    interpreted as part of the same string. So use line breaks like this 
    to keep your lines short.
    
    Args:
        par1: a number
        par2: a second number
    
    Returns:
        The result of some operation on our input



In [222]:
help(len)

Help on built-in function len in module builtins:

len(obj, /)
    Return the number of items in a container.



In the Jupyter environment, you can also get the docstring by pressing Shift+Tab on a function.

Here I'm using my docstrings in the so-called Google format. There are other options, as you can read about online:

https://stackoverflow.com/questions/3898572/what-are-the-most-common-python-docstring-formats https://betterprogramming.pub/3-different-docstring-formats-for-python-d27be81e0d68

For more on how to write good docstrings, check out the PEP conventions:

https://peps.python.org/pep-0257/